# Flask Exercise

The goal of the exercise is to try and practice on Flask, is a step by step project. First of all create a simple flask application.

In [2]:
from flask import Flask, jsonify, request, abort, redirect

In [ ]:
app = Flask("BigDive")

@app.route('/')
def index():
    return "Welcome!"

We want to create a simple application to manage a list of blogs! Will reply only with JSON! First of all create a list and call it **blogs**.

Every single blog has:

- id (integer)
- title  (string)
- body   (string)
- author (dictionary with firstname and lastname)

Each blog is a **dictionary** saved inside the list.

So, create a blogs list with a single blog inside!

In [ ]:
blogs = [ {'id': 1, 
           'title': 'Hello World', 
           'body': 'Welcome to my Blog!', 
           'author': 'alexcomu'}
        ]

Now edit the **index** function, the index function should return the list of blogs!

In [ ]:
app = Flask("BigDive")

@app.route('/')
def index():
    return jsonify(
        blogs=blogs
    ), 200

Ok we can start, create a new URL the show the single blog given the ID. For example:

    http://localhost:5000/blog/1
    
Should reply with something like:

    {
        "id":1,
        "title":"Hello World",
        "body":"Welcome to my Blog!",
        "author": "alexcomu"
    }

If the **blogid** doesn't exists, you should return a **404**.

In [ ]:
app = Flask("BigDive")

@app.route('/')
def index():
    return jsonify(
        blogs=blogs
    ), 200

def search_id(blog_list, blogid):
    for index, blog in enumerate(blog_list):
        if blogid == blog['id']:
            return index
    raise Exception
    
@app.route('/blog/<int:blogid>')
def blog(blogid):
    try:
        index = search_id(blogs, blogid)
        blog = blogs[index]
    except:
        abort(404)
    return jsonify(blog), 200


**Great!**

Now, as you can see the 404 is an HTML page. Edit your code and change the response of the 404 error to a Json!

In [ ]:
app = Flask("BigDive")

@app.route('/')
def index():
    return jsonify(
        blogs=blogs
    ), 200

def search_id(blog_list, blogid):
    for index, blog in enumerate(blog_list):
        if blogid == blog['id']:
            return index
    raise Exception

@app.route('/blog/<int:blogid>')
def blog(blogid):
    try:
        blog = blogs[search_id(blogs, blogid)]
    except:
        abort(404)
    return jsonify(blog), 200

@app.errorhandler(404)
def not_found(error):
    return jsonify(error="Not Found"), 404


Perfect! Now we would like to be able to create new blog! So, create a new URL like:

    http://localhost:5000/create
    
This URL must accept **POST** requests only and should receive all the information required to create the blog! If some information is missing, reply with an error **400 Bad request**.

**ATTENTION** the **id** field should be automatic generated by the system!

If everything is ok, add the blog to the list and return the new blog created!

In [ ]:
app = Flask("BigDive")
blogs = []

@app.route('/')
def index():
    return jsonify(
        blogs=blogs
    ), 200

def search_id(blog_list, blogid):
    for index, blog in enumerate(blog_list):
        if blogid == blog['id']:
            return index
    raise Exception

def generate_id():
    try:
        return blogs[-1]['id'] +1
    except IndexError:
        return 1
    
@app.route('/blog/<int:blogid>')
def blog(blogid):
    try:
        blog = blogs[search_id(blogs, blogid)]
    except:
        abort(404)
    return jsonify(blog), 200

@app.errorhandler(404)
def not_found(error):
    return jsonify(error="Not Found"), 404

@app.errorhandler(400)
def bad_request(error):
    return jsonify(error="Bad Request"), 400

@app.route('/create', methods=['post'])
def create():
    requested_params = ('title', 'body', 'author')
    if not all (key in request.form for key in (requested_params)):
        abort(400)
    blog = request.form.to_dict(flat=True)
    blog['id'] = generate_id()
    blogs.append(blog)
    return jsonify(blog), 200


Great! Now try to create a URL to delete a blog and to update a blog:

    http://localhost:5000/edit/<int:blogid> (HTTP PUT) - Return blog 
    http://localhost:5000/delete/<int:blogid> (HTTP DELETE) - Redirect on "/"

In [ ]:
app = Flask("BigDive")
blogs = []

@app.route('/')
def index():
    return jsonify(
        blogs=blogs
    ), 200

def search_id(blog_list, blogid):
    for index, blog in enumerate(blog_list):
        if blogid == blog['id']:
            return index
    raise Exception

def generate_id():
    try:
        return blogs[-1]['id'] +1
    except IndexError:
        return 1
    
@app.route('/blog/<int:blogid>')
def blog(blogid):
    try:
        blog = blogs[search_id(blogs, blogid)]
    except:
        abort(404)
    return jsonify(blog), 200

@app.errorhandler(404)
def not_found(error):
    return jsonify(error="Not Found"), 404

@app.errorhandler(400)
def bad_request(error):
    return jsonify(error="Bad Request"), 400

@app.route('/create', methods=['post'])
def create():
    requested_params = ('title', 'body', 'author')
    if not all (key in request.form for key in (requested_params)):
        abort(400)
    blog = request.form.to_dict(flat=True)
    blog['id'] = generate_id()
    blogs.append(blog)
    return jsonify(blog), 200

@app.route('/delete/<int:blogid>', methods=['delete'])
def delete(blogid):
    try:
        index = search_id(blogs, blogid)
    except:
        abort(404)
    blogs.remove(blogs[index])
    return redirect('/')    

@app.route('/edit/<int:blogid>', methods=['put'])
def edit(blogid):
    try:
        index = search_id(blogs, blogid)
    except:
        abort(404)
    requested_params = ('title', 'body', 'author')
    if not all (key in request.form for key in (requested_params)):
        abort(400)
    blogs[index] = request.form.to_dict(flat=True)
    blogs[index]['id'] = blogid
    return jsonify(request.form.to_dict(flat=True)), 200
app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Nov/2017 15:16:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2017 15:16:43] "GET /blog/1 HTTP/1.1" 404 -
127.0.0.1 - - [21/Nov/2017 15:17:44] "GET / HTTP/1.1" 200 -
Traceback (most recent call last):
  File "/usr/local/Cellar/python/2.7.13/Frameworks/Python.framework/Versions/2.7/lib/python2.7/SocketServer.py", line 290, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/local/Cellar/python/2.7.13/Frameworks/Python.framework/Versions/2.7/lib/python2.7/SocketServer.py", line 318, in process_request
    self.finish_request(request, client_address)
  File "/usr/local/Cellar/python/2.7.13/Frameworks/Python.framework/Versions/2.7/lib/python2.7/SocketServer.py", line 331, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/local/Cellar/python/2.7.13/Frameworks/Python.framework/Versions/2.7/lib/python2.7/SocketServer.py", line 654, in _

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 65106)
----------------------------------------
